## **Components (Migrate from Stable Diffusion)**
Quick migration mental model (old → new)

1. U-Net → Transformer (MMDiT-X)

2. 1× CLIP → 2× CLIP + 1× T5-XXL

3. Same VAE role (encode/decode latents)

4. DDIM/DPM schedulers → FlowMatch-Euler/Heun (+ shift)

5. CFG only → CFG + optional SLG


### **1. Transformer**

In [1]:
from diffusers import SD3Transformer2DModel, TorchAoConfig
model_id = "stabilityai/stable-diffusion-3.5-medium"
ao_cfg = TorchAoConfig("int4_weight_only")

transformer = SD3Transformer2DModel.from_pretrained(
    model_id,
    subfolder="transformer",
    quantization_config= ao_cfg,
)

Overriding `torch_dtype` with `torch_dtype=torch.bfloat16` due to requirements of `torchao` to enable model loading in different precisions. Pass your own `torch_dtype` to specify the dtype of the remaining non-linear layers, or pass torch_dtype=torch.bfloat16, to remove this warning.


NotImplementedError: Could not run 'aten::_convert_weight_to_int4pack' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::_convert_weight_to_int4pack' is only available for these backends: [CUDA, Meta, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradMAIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastMTIA, AutocastMAIA, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncT

In [ ]:
from diffusers import StableDiffusion3Pipeline